# YASA feature extraction
This notebook attempts to run the YASA out-of-the-box model on our seals and examines the accuracy to see if this capstone project is even needed at all

In [ ]:
import yasa
import mne
import pandas as pd
import numpy as np
import pytz
import datetime

In [ ]:
path_to_edf = '../data/01_processed-data/test12_Wednesday_05_ALL_PROCESSED.edf'
raw = mne.io.read_raw_edf(path_to_edf, include=['EEG_ICA5'], preload=False)

In [ ]:
info = raw.info
sfreq = info['sfreq']
edf_start_time = info['meas_date']
# Define the PST timezone
pst_timezone = pytz.timezone('America/Los_Angeles')
# Convert to datetime object in PST
if isinstance(edf_start_time, datetime.datetime):
    # If it's already a datetime object, just replace the timezone
    recording_start_datetime = edf_start_time.replace(tzinfo=None).astimezone(pst_timezone)
    # for some reason using .replace(tzinfo=...) does weird shit - offsets based of LMT instead of UTC and gets confusing
    # recording_start_datetime = edf_start_time.replace(tzinfo=pst_timezone)
elif isinstance(edf_start_time, (int, float)):
    # Convert timestamp to datetime in PST
    recording_start_datetime = pst_timezone.localize(datetime.datetime.fromtimestamp(edf_start_time))

# Load Labels

In [ ]:
labels_df = pd.read_csv('../data/02_annotated-data/test12_Wednesday_06_Hypnogram_JKB_1Hz.csv')
labels_df['R.Time'] = pd.to_datetime(labels_df['R.Time']).dt.tz_localize('America/Los_Angeles')


In [ ]:
start_seconds = int((labels_df['R.Time'].iloc[0] - recording_start_datetime).total_seconds())
end_seconds = start_seconds + 24 * 60 * 60

In [ ]:
raw.crop(start_seconds, end_seconds)
print(len(raw.get_data('EEG_ICA5')[0]) / 500)
print(len(labels_df))

# Sleep Staging

In [ ]:
sleep_stage = yasa.SleepStaging(raw, eeg_name='EEG_ICA5')

In [ ]:
sleep_stage.fit()
hypno = sleep_stage.predict()

### Map our labels to theirs

In [ ]:
labelmap = {
    'W':'Active Waking',
    #'': 'Quiet Waking',
    'N1': 'Drowsiness',
    'N2': 'SWS',
    'N3': 'SWS',
    'R': 'REM'
}